In [60]:
import pandas as pd

log_path = '../data/HDFS.log'
with open(log_path, 'r') as file:
    lines = file.readlines()

logs = pd.DataFrame(lines, columns=['log'])
logs.head()


,log
0,081111 032233 34 INFO dfs.FSNamesystem: BLOCK*...
1,081110 123023 11088 INFO dfs.DataNode$PacketRe...
2,081111 072459 34 INFO dfs.FSNamesystem: BLOCK*...
3,081111 023014 27 INFO dfs.FSNamesystem: BLOCK*...
4,081111 030004 18183 INFO dfs.DataNode$DataXcei...


In [61]:
import regex as re

# Функция для извлечения block_id
def extract_block_id(log_line):
    match = re.search(r'blk_-?\d+', log_line)
    return match.group(0) if match else None

logs['block_id'] = logs['log'].apply(extract_block_id)

logs.dropna(subset=['block_id'], inplace=True)
all_logs = logs['log'].unique()

logs.head()


,log,block_id
0,081111 032233 34 INFO dfs.FSNamesystem: BLOCK*...,blk_-5327725140452543501
1,081110 123023 11088 INFO dfs.DataNode$PacketRe...,blk_-8773898409706445615
2,081111 072459 34 INFO dfs.FSNamesystem: BLOCK*...,blk_-4329641471754524122
3,081111 023014 27 INFO dfs.FSNamesystem: BLOCK*...,blk_-390921607507384009
4,081111 030004 18183 INFO dfs.DataNode$DataXcei...,blk_-2967608524545562422


In [62]:
sessions = logs.groupby('block_id')['log'].apply(list).reset_index(name='sequence')

sessions.head()

,block_id,sequence
0,blk_-1000002529962039464,[081111 094454 25868 INFO dfs.DataNode$PacketR...
1,blk_-100000266894974466,[081110 020141 6105 INFO dfs.DataNode$PacketRe...
2,blk_-1000054577281647820,[081110 012531 5179 INFO dfs.DataNode$PacketRe...
3,blk_-1000245396392748444,[081110 002154 4817 INFO dfs.DataNode$PacketRe...
4,blk_-1000285592763698141,[081110 162157 13111 INFO dfs.DataNode$DataXce...


In [63]:

log2idx = {log: idx for idx, log in enumerate(all_logs)}

def fast_encode(seq):
    return [log2idx[log] for log in seq if log in log2idx]

sessions['encoded'] = sessions['sequence'].apply(fast_encode)

sessions.head()

,block_id,sequence,encoded
0,blk_-1000002529962039464,[081111 094454 25868 INFO dfs.DataNode$PacketR...,[242059]
1,blk_-100000266894974466,[081110 020141 6105 INFO dfs.DataNode$PacketRe...,[261783]
2,blk_-1000054577281647820,[081110 012531 5179 INFO dfs.DataNode$PacketRe...,[109530]
3,blk_-1000245396392748444,[081110 002154 4817 INFO dfs.DataNode$PacketRe...,[3086]
4,blk_-1000285592763698141,[081110 162157 13111 INFO dfs.DataNode$DataXce...,[86956]


In [64]:
import numpy as np

window_size = 5  

X = []
y = []

for encoded_seq in sessions['encoded']:
    if len(encoded_seq) > window_size:
        for i in range(len(encoded_seq) - window_size):
            X.append(encoded_seq[i:i+window_size])
            y.append(encoded_seq[i+window_size])

X = np.array(X)
y = np.array(y)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print((sessions['encoded'].apply(len) > 5).sum())

X shape: (200, 5)
y shape: (200,)
80


In [65]:
import torch
from torch.utils.data import Dataset, DataLoader

class LogDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = LogDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [66]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTMLogModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super(LSTMLogModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.lstm(x)
        last_hidden = output[:, -1, :]
        out = self.fc(last_hidden)
        return out

vocab_size = len(log2idx)
model = LSTMLogModel(vocab_size).to(device)

In [67]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        logits = model(batch_x)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

Epoch 1/5 - Loss: 12.8286
Epoch 2/5 - Loss: 12.7237
Epoch 3/5 - Loss: 12.6111
Epoch 4/5 - Loss: 12.4735
Epoch 5/5 - Loss: 12.2549


In [15]:
import random
model.eval()

idx = random.randint(0, len(X) - 1)
test_seq = torch.tensor(X[idx]).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(test_seq)
    predicted_token = torch.argmax(output, dim=1).item()
    top_k = 5  
    topk_indices = torch.topk(output, k=top_k, dim=1).indices[0].tolist()

# Восстановим текст логов по индексам
input_logs = [log for log, ix in log2idx.items() if ix in X[idx]]
predicted_log = [log for log, ix in log2idx.items() if ix == predicted_token][0]
real_log = [log for log, ix in log2idx.items() if ix == y[idx]][0]

print(" Входная последовательность:")
for log in input_logs:
    print("-", log.strip())

print("\nПредсказанный следующий лог :")
print("-", predicted_log.strip())

print("\n Реальный лог:")
print("-", real_log.strip())

print("\n  топ 5 предсказаний:")
for i, ix in enumerate(topk_indices):
    log_txt = [log for log, log_ix in log2idx.items() if log_ix == ix][0]
    print(f"{i+1}. {log_txt.strip()}")

 Входная последовательность:
- 081111 100151 25905 INFO dfs.DataNode$DataXceiver: 10.251.203.166:50010 Served block blk_134407113958772458 to /10.251.127.47
- 081111 100154 25941 INFO dfs.DataNode$DataXceiver: 10.251.203.166:50010 Served block blk_134407113958772458 to /10.250.10.100
- 081111 110418 19 INFO dfs.FSDataset: Deleting block blk_134407113958772458 file /mnt/hadoop/dfs/data/current/subdir33/blk_134407113958772458
- 081111 100153 25935 INFO dfs.DataNode$DataXceiver: 10.251.203.166:50010 Served block blk_134407113958772458 to /10.250.6.214
- 081111 100152 26330 INFO dfs.DataNode$DataXceiver: 10.251.90.81:50010 Served block blk_134407113958772458 to /10.251.110.196

Предсказанный следующий лог :
- 081111 100152 25932 INFO dfs.DataNode$DataXceiver: 10.251.203.166:50010 Served block blk_134407113958772458 to /10.251.65.237

 Реальный лог:
- 081111 100152 25932 INFO dfs.DataNode$DataXceiver: 10.251.203.166:50010 Served block blk_134407113958772458 to /10.251.65.237

  топ 5 предск

In [16]:
def evaluate_top_k(model, X, y, log2idx, top_k_list=[1, 3, 5]):
    model.eval()
    results = {k: 0 for k in top_k_list}
    total = len(X)

    with torch.no_grad():
        for i in range(total):
            x_input = torch.tensor(X[i]).unsqueeze(0).to(device)  # shape: [1, seq_len]
            target = y[i]

            logits = model(x_input)
            for k in top_k_list:
                topk = torch.topk(logits, k=k, dim=1).indices[0].tolist()
                if target in topk:
                    results[k] += 1

    for k in top_k_list:
        acc = results[k] / total * 100
        print(f" Accuracy@{k}: {acc:.2f}% ({results[k]}/{total})")

evaluate_top_k(model, X, y, log2idx)

 Accuracy@1: 54.50% (109/200)
 Accuracy@3: 67.50% (135/200)
 Accuracy@5: 76.50% (153/200)


In [45]:
label_path = '../data/anomaly_label.csv'
labels_df = pd.read_csv(label_path)

print(labels_df.head())
print(labels_df.columns)

                    BlockId    Label
0  blk_-1608999687919862906   Normal
1   blk_7503483334202473044   Normal
2  blk_-3544583377289625738  Anomaly
3  blk_-9073992586687739851   Normal
4   blk_7854771516489510256   Normal
Index(['BlockId', 'Label'], dtype='object')


In [46]:
labels_df.columns = ['block_id', 'label']

labels_df['block_id'] = labels_df['block_id'].astype(str)
sessions['block_id'] = sessions['block_id'].astype(str)

sessions = sessions.merge(labels_df, on='block_id', how='inner')

# Преобразуем метки в числа: Normal = 0, Anomaly = 1
sessions['label'] = sessions['label'].map({'Normal': 0, 'Anomaly': 1})

sessions.head()

,block_id,sequence,label
0,blk_-1000002529962039464,[081111 094454 25868 INFO dfs.DataNode$PacketR...,0
1,blk_-100000266894974466,[081110 020141 6105 INFO dfs.DataNode$PacketRe...,0
2,blk_-1000054577281647820,[081110 012531 5179 INFO dfs.DataNode$PacketRe...,0
3,blk_-1000245396392748444,[081110 002154 4817 INFO dfs.DataNode$PacketRe...,0
4,blk_-1000285592763698141,[081110 162157 13111 INFO dfs.DataNode$DataXce...,0


In [50]:
# Преобразуем последовательности логов в текст
sessions['log_text'] = sessions['sequence'].apply(lambda seq: ' '.join(seq))

sessions[['log_text', 'label']].head()

,log_text,label
0,081111 094454 25868 INFO dfs.DataNode$PacketRe...,0
1,081110 020141 6105 INFO dfs.DataNode$PacketRes...,0
2,081110 012531 5179 INFO dfs.DataNode$PacketRes...,0
3,081110 002154 4817 INFO dfs.DataNode$PacketRes...,0
4,081110 162157 13111 INFO dfs.DataNode$DataXcei...,0


In [51]:
from sklearn.model_selection import train_test_split

train_sessions, test_sessions = train_test_split(
    sessions,
    test_size=0.2,
    stratify=sessions['label'],
    random_state=42
)

In [52]:
normal = train_sessions[train_sessions['label'] == 0].sample(n=6640, random_state=42)
anomaly = train_sessions[train_sessions['label'] == 1]  # их 6640 всего

train_balanced = pd.concat([normal, anomaly]).sample(frac=1, random_state=42).reset_index(drop=True)

In [53]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#  обучающие — сбалансированные
tokenized_train = tokenizer(
    train_balanced['log_text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# тестовые — реальные
tokenized_test = tokenizer(
    test_sessions['log_text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

print(" Train input_ids shape:", tokenized_train['input_ids'].shape)
print(" Test input_ids shape:", tokenized_test['input_ids'].shape)

 Train input_ids shape: torch.Size([11952, 128])
 Test input_ids shape: torch.Size([54595, 128])


In [54]:
from torch.utils.data import Dataset, DataLoader
import torch

# Создаём класс Dataset для логов
class LogBERTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

# Превращаем метки в тензоры
train_labels = torch.tensor(train_balanced['label'].values)
test_labels = torch.tensor(test_sessions['label'].values)

# Создаём датасеты
train_dataset = LogBERTDataset(tokenized_train, train_labels)
test_dataset = LogBERTDataset(tokenized_test, test_labels)

# Загружаем батчи
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [55]:
from transformers import BertForSequenceClassification

# BERT с классификацией на 2 класса
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2  # 0: Normal, 1: Anomaly
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [56]:
from torch.optim import AdamW
from tqdm import tqdm  

optimizer = AdamW(model.parameters(), lr=2e-5)
weight = torch.tensor([1.0, 10.0]).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=weight)

In [32]:

#epochs = 3  

#for epoch in range(epochs):
#    model.train()
#    total_loss = 0

#    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
#    for batch in loop:
#        input_ids = batch['input_ids'].to(device)
#        attention_mask = batch['attention_mask'].to(device)
#        labels = batch['labels'].to(device)
#
#        optimizer.zero_grad()
#        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#        loss = outputs.loss
#        loss.backward()
#        optimizer.step()

#        total_loss += loss.item()
#        loop.set_postfix(loss=loss.item())

#    avg_loss = total_loss / len(train_loader)
#    print(f" Epoch {epoch+1} — Average Loss: {avg_loss:.4f}")

    
    #был весь  датасет, пришлось уменьшить до 12000 сессий  чтобы ускорить 

Epoch 1/3:   5%|▍         | 634/13649 [19:18<6:36:22,  1.83s/it, loss=0.246] 


KeyboardInterrupt: 

In [57]:
epochs = 3


for epoch in range(epochs):
    model.train()
    total_loss = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f" Epoch {epoch+1} — Average Loss: {avg_loss:.4f}")

Epoch 1/3: 100%|██████████| 1494/1494 [23:13<00:00,  1.07it/s, loss=0.32] 


 Epoch 1 — Average Loss: 0.4170


Epoch 2/3: 100%|██████████| 1494/1494 [45:36<00:00,  1.83s/it, loss=0.313]   


 Epoch 2 — Average Loss: 0.4047


Epoch 3/3: 100%|██████████| 1494/1494 [21:36<00:00,  1.15it/s, loss=0.378] 

 Epoch 3 — Average Loss: 0.4012


In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Метрики
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f" Accuracy:  {accuracy:.4f}")
print(f" Precision: {precision:.4f}")
print(f" Recall:    {recall:.4f}")
print(f" F1 Score:  {f1:.4f}")
print("\n Confusion Matrix:")
print(conf_matrix)

Evaluating: 100%|██████████| 6825/6825 [19:39<00:00,  5.79it/s]

 Accuracy:  0.0243
 Precision: 0.0243
 Recall:    1.0000
 F1 Score:  0.0475

 Confusion Matrix:
[[    0 53267]
 [    0  1328]]
